# Imports

In [7]:
import pandas as pd
import numpy as np
import tweepy as tw
import unicodecsv as csv
import re
import operator

# Extract tweets using search and stream API, store in .csv 

In [2]:
ACCESS_TOKEN = "1187499625646870534-DuNGIM7uAsnPrUjmRaXVZd5bFGS7Ng"
ACCESS_TOKEN_SECRET = "ijlw6NuIJjt48p2bHjhUPq7joXhF3dRUuDqHJIkHsfT1w"
CONSUMER_KEY = "P15uXJnpbXUxY0t9VaLL0Pvps"
CONSUMER_SECRET = "dC8j23a1mC010eK38YzlT143G9b9dMEJiaR7K0DWVnIANlJIdK"

In [3]:
auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tw.API(auth, wait_on_rate_limit=True)

### Using search api

In [11]:
def get_tweets(listOfTweets,query,numOfTweets):
    for tweet in tw.Cursor(api.search, q=query, tweet_mode="extended").items(numOfTweets):
        dict_ = {'Screen_Name': tweet.user.screen_name,
                'User_Name': tweet.user.name,
                'Tweet_Created At': str(tweet.created_at),
                'User_Location': str(tweet.user.location),
                'Tweet_Coordinates': str(tweet.coordinates),
                'Retweet_Count': str(tweet.retweet_count),
                'Retweeted': str(tweet.retweeted),
                'Phone_Type': str(tweet.source),
                'Favorite_Count': str(tweet.favorite_count),
                'Favorited': str(tweet.favorited),
                'Replied': str(tweet.in_reply_to_status_id_str),
                'Extended_Tweet': str(tweet.full_text)
                }
        try:
            dict_['retweet_text']  = tweet.retweeted_status.full_text
        except:
            dict_['retweet_text'] = 'None'
        listOfTweets.append(dict_)   
    return listOfTweets

In [12]:
listOfTweets0 = []
keyword = "Dalhousie University"
numOfTweets = 800
get_tweets(listOfTweets0, keyword, numOfTweets)

listOfTweets1 = []
keyword = ["Canada"]
numOfTweets = 800
get_tweets(listOfTweets1, keyword, numOfTweets)

listOfTweets2 = []
keyword = ["University"]
numOfTweets = 800
get_tweets(listOfTweets2, keyword, numOfTweets)

listOfTweets3 = []
keyword = ["Halifax"]
numOfTweets = 800
get_tweets(listOfTweets3, keyword, numOfTweets)

listOfTweets4 = []
keyword = ["Canada Education"]
numOfTweets = 800
get_tweets(listOfTweets4, keyword, numOfTweets)

listOfTweets = []
listOfTweets = listOfTweets0 + listOfTweets1 + listOfTweets2 + listOfTweets3 + listOfTweets4

In [13]:
print(len(listOfTweets))
print(len(listOfTweets0))
print(len(listOfTweets1))
print(len(listOfTweets2))
print(len(listOfTweets3))
print(len(listOfTweets4))

3608
408
800
800
800
800


In [21]:
keys = listOfTweets[0].keys()
with open('tweets.csv', 'wb') as file:
    dict_writer = csv.DictWriter(file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(listOfTweets)

# Clean tweets and store in clean_tweets.csv

In [6]:
df_tweet = pd.read_csv("tweets.csv")

In [7]:
def replace_func(row):
    if(row['retweet_text'] == 'None'):
        return row['Extended Tweet']
    else:
        return row['retweet_text']

In [8]:
df_tweet['Tweet'] = df_tweet.apply(lambda row : replace_func(row),axis = 1)

In [9]:
df_tweet = df_tweet.drop(['Extended Tweet','retweet_text'],axis = 1)

In [10]:
def clean_tweets(t):
    t = re.sub(r"(?:\@|https?\://)\S+", "", t)  #remove user name from tweets and URLs
    t = t.encode('ascii', 'ignore').decode('ascii') #remove emoticons
    t = re.sub(r"[^a-zA-Z0-9]+", ' ', t) 
    t = t.lower()
    t = t.replace("'s","") #cases Dalhousie University's, Halifax's
    t = t.replace(r"\n", " ")
    return t

In [39]:
df_tweet['Tweet'] = df_tweet['Tweet'].apply(lambda tweet:clean_tweets(tweet))
df_tweet['Tweet'].replace(' ', np.nan, inplace=True)
print("Total empty tweets:",df_tweet['Tweet'].isnull().sum())
df_tweet.dropna(subset=['Tweet'], inplace=True)
print("Total empty tweets after cleaning:",df_tweet['Tweet'].isnull().sum())

Total empty tweets: 22
Total empty tweets after cleaning: 0


In [42]:
df_tweet.to_csv(r'clean_tweets.csv')

In [48]:
df_tweet.shape

(3572, 12)

In [44]:
#At this point make decision if you want just one column...of tweet

# Extract news using news API, store in .csv

In [89]:
from newsapi import NewsApiClient

In [90]:
api = NewsApiClient(api_key='e0fba18a6f1d4feba8427fc69e0637dc')

In [105]:
ALL_ARTICLES = []

all_articles = api.get_everything(q='University',
                                      from_param='2019-10-05',
                                      to='2019-11-02',
                                      language='en',
                                      sort_by='relevancy',
                                 page_size = 100)
ALL_ARTICLES += all_articles['articles']

all_articles = api.get_everything(q='Canada',
                                      from_param='2019-10-05',
                                      to='2019-11-02',
                                      language='en',
                                      sort_by='relevancy',
                                 page_size = 100)
ALL_ARTICLES += all_articles['articles']

all_articles = api.get_everything(q='Dalhousie University',
                                      from_param='2019-10-05',
                                      to='2019-11-02',
                                      language='en',
                                      sort_by='relevancy',
                                 page_size = 100)
ALL_ARTICLES += all_articles['articles']

all_articles = api.get_everything(q='Halifax',
                                      from_param='2019-10-05',
                                      to='2019-11-02',
                                      language='en',
                                      sort_by='relevancy',
                                 page_size = 100)
ALL_ARTICLES += all_articles['articles']

all_articles = api.get_everything(q='Canada Education',
                                      from_param='2019-10-05',
                                      to='2019-11-02',
                                      language='en',
                                      sort_by='relevancy',
                                 page_size = 100)
ALL_ARTICLES += all_articles['articles']


In [106]:
len(ALL_ARTICLES)

499

In [120]:
keys = ALL_ARTICLES[0].keys()
with open('news.csv', 'wb') as file:
    dict_writer = csv.DictWriter(file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(ALL_ARTICLES)

# Clean news and store in clean_news.csv

In [14]:
def clean_news(n):
    n = re.sub('\[[^]]*\]', '', n)
    n = re.sub(r"(?:\@|https?\://)\S+", "", n)  #remove user name from tweets and URLs
    n = n.encode('ascii', 'ignore').decode('ascii') #remove emoticons
    n = re.sub(r"[^a-zA-Z0-9]+", ' ', n) 
    n = n.lower()
    n = n.replace("'s","") #cases Dalhousie University's, Halifax's
    n = n.replace(r"\n", " ")
    n = n.replace(r"\r", " ")
    return n

In [15]:
df_news = pd.read_csv("news.csv")
df_news

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'engadget', 'name': 'Engadget'}",Christine Fisher,Howard University and Amazon want to diversify...,Howard University and Amazon Studios are worki...,https://www.engadget.com/2019/10/11/amazon-how...,https://o.aolcdn.com/images/dims?thumbnail=120...,2019-10-11T14:23:00Z,The new program will send students to Los Ange...
1,"{'id': 'the-verge', 'name': 'The Verge'}",Russell Brandom,Mark Zuckerberg took on China in a speech defe...,In a speech at Georgetown University on Octobe...,https://www.theverge.com/2019/10/17/20919464/m...,https://cdn.vox-cdn.com/thumbor/1s2MYUZgWfbZFX...,2019-10-17T18:45:21Z,Is that the internet that we want?\r\nIn a spe...
2,"{'id': 'cnn', 'name': 'CNN'}","Julia Jones, CNN",Penn State investigates new allegations agains...,Penn State University is investigating a new a...,https://www.cnn.com/2019/10/31/us/sandusky-new...,https://cdn.cnn.com/cnnnext/dam/assets/1910311...,2019-10-31T21:13:28Z,(CNN)Penn State University is investigating a ...
3,"{'id': 'mashable', 'name': 'Mashable'}",Danica D'Souza,This 3D food printer cooks its prints with a l...,Researchers at Columbia University are leading...,https://mashable.com/video/3d-food-printing-la...,https://mondrian.mashable.com/2019%252F10%252F...,2019-10-15T20:52:06Z,NaN
4,"{'id': 'the-verge', 'name': 'The Verge'}",Zoe Schiffer,Mark Zuckerberg on lies in political ads: ‘I d...,Facebook’s CEO Mark Zuckerberg rallied people ...,https://www.theverge.com/2019/10/17/20919223/m...,https://cdn.vox-cdn.com/thumbor/sIycmXYlUtgRYI...,2019-10-17T18:39:24Z,Facebooks CEO rallies people around the First ...
5,"{'id': None, 'name': 'Gizmodo.com'}",Ryan F. Mandelbaum,Physicists Win Nobel Prize for Finding Exoplan...,The 2019 Physics Nobel Prize will be awarded h...,https://gizmodo.com/physicists-win-nobel-prize...,https://i.kinja-img.com/gawker-media/image/upl...,2019-10-08T14:54:00Z,The 2019 Physics Nobel Prize will be awarded h...
6,"{'id': None, 'name': 'Gizmodo.com'}","Tom McKay on Earther, shared by Tom McKay to G...",Report: University of Iowa Faculty Told Not to...,University of Iowa officials have told faculty...,https://earther.gizmodo.com/report-university-...,https://i.kinja-img.com/gawker-media/image/upl...,2019-10-05T23:00:00Z,University of Iowa officials have told faculty...
7,"{'id': 'cnn', 'name': 'CNN'}","Amir Vera, CNN",Lori Loughlin's daughters are no longer enroll...,Lori Loughlin's daughters are no longer enroll...,https://www.cnn.com/2019/10/21/us/college-admi...,https://cdn.cnn.com/cnnnext/dam/assets/1907130...,2019-10-22T00:14:18Z,(CNN)Lori Loughlin's daughters are no longer e...
8,"{'id': 'the-new-york-times', 'name': 'The New ...",Anna Schaverien,Oxford Professor Is Accused of Selling Ancient...,An investigation found that Bible fragments in...,https://www.nytimes.com/2019/10/16/world/europ...,https://static01.nyt.com/images/2019/10/16/wor...,2019-10-16T16:22:47Z,"But Stephen Rouse, an Oxford spokesman, said W..."
9,"{'id': 'the-new-york-times', 'name': 'The New ...",Billy Witz,"Beer, Here: Merchandising of College Sports Le...",The University of Louisiana at Lafayette and m...,https://www.nytimes.com/2019/11/01/sports/ncaa...,https://static01.nyt.com/images/2019/11/02/spo...,2019-11-02T04:20:27Z,"Early in the summer of 2015, Knott received a ..."


In [16]:
df_news['content'] = df_news['content'].apply(lambda news:clean_news(str(news)))
df_news['title'] = df_news['title'].apply(lambda news:clean_news(str(news)))
df_news['description'] = df_news['description'].apply(lambda news:clean_news(str(news)))

In [17]:
df_news['content'].replace(' ', np.nan, inplace=True)
print("Total empty tweets:",df_news['content'].isnull().sum())
df_news.dropna(subset=['content'], inplace=True)
print("Total empty tweets after cleaning:",df_news['content'].isnull().sum())
df_news = df_news[df_news.content != 'nan']

df_news['title'].replace(' ', np.nan, inplace=True)
print("Total empty tweets:",df_news['title'].isnull().sum())
df_news.dropna(subset=['title'], inplace=True)
print("Total empty tweets after cleaning:",df_news['title'].isnull().sum())
df_news = df_news[df_news.title != 'nan']

df_news['description'].replace(' ', np.nan, inplace=True)
print("Total empty tweets:",df_news['description'].isnull().sum())
df_news.dropna(subset=['description'], inplace=True)
print("Total empty tweets after cleaning:",df_news['description'].isnull().sum())
df_news = df_news[df_news.description != 'nan']


Total empty tweets: 2
Total empty tweets after cleaning: 0
Total empty tweets: 0
Total empty tweets after cleaning: 0
Total empty tweets: 0
Total empty tweets after cleaning: 0


C:\Users\darpa\Anaconda3\lib\site-packages\pandas\core\generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\darpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
df_news.to_csv(r'clean_news.csv')

## Merge news and tweet

In [49]:
merge_df = pd.concat([df_tweet["Tweet"],df_news["content"]],axis = 0) # here you can concate news 'title' or 'description' if required...
merge_df = pd.DataFrame(merge_df)
merge_df.columns = ["data"]
merge_df

,data
0,last week dr jonathon fowles eim canada chair ...
1,building on dalhousie university campus closed...
2,industry cult legend rick simpson on big pharm...
3,industry cult legend rick simpson on big pharm...
4,industry cult legend rick simpson on big pharm...
5,our next postgraduate open day will take place...
6,industry cult legend rick simpson on big pharm...
7,industry cult legend rick simpson on big pharm...
8,industry cult legend rick simpson on big pharm...
9,open dialogue edward snowden live from russia ...


In [50]:
merge_df.to_csv(r'merged_data.csv')

# Data processing using pyspark 

In [1]:
from pyspark import SparkContext
sc = SparkContext( master = "spark://192.168.2.22:7077") #IP and Port of where master is running...

Keywords:
[“education”,
“Canada”,
“university”,
“dalhousie”,
“expensive”,
“good school” ,
“good schools”,
“bad school” ,
“bad schools” ,
“poor school”, 
“poor schools”,
“faculty”,
“computer science”,
“graduate”]

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
single_keys = ["education","canada","university","dalhousie","expensive","faculty","graduate"]
double_keys = ["good school","good schools","bad school","bad schools","poor school","poor schools","computer science"]

In [4]:
df_spark = sqlContext.read.load("merged_data.csv",
                               format = 'com.databricks.spark.csv',
                               header = 'true',
                               inferSchema = 'true').select('data')

In [5]:
def get_keyval(row):
    text = row.data
    words = text.split(" ")
    return [[w,1] for w in words if w in single_keys] #here put condition if w.equals("a" or "b" or "c")

In [8]:
mapped_rdd = df_spark.rdd.flatMap(lambda row:get_keyval(row))
counts_rdd = mapped_rdd.reduceByKey(operator.add)
word_count = counts_rdd.collect()

In [9]:
word_count

[('canada', 1449),
 ('dalhousie', 348),
 ('university', 1267),
 ('graduate', 16),
 ('faculty', 13),
 ('education', 818),
 ('expensive', 5)]

In [10]:
def bigram(words):
    bigrams = []
    for i in range(len(words) - 1):
        word = (words[i]+" "+words[i+1])
        if(word in double_keys):# check "good schools","computer science" etc...if ele IN list_of_search
            bigrams.append((tuple(words[i:i+2]), 1))
    return bigrams


bigrams = df_spark.rdd.map(lambda line: line.data.split(" ")).flatMap(bigram)
freq_bigrams = bigrams.reduceByKey(lambda x, y: x + y).map(lambda x : (x[1], x[0])).sortByKey(False)

In [11]:
freq_lst = freq_bigrams.collect()

In [12]:
freq_lst

[(5, ('computer', 'science'))]

In [13]:
sc.stop()

In [19]:
f = open('output.txt',"a+")

In [20]:
for item in word_count:
    f.write('{}:{}'.format(item[0],item[1]))
    f.write("\n")

In [21]:
for item in freq_lst:
    f.write('{}:{}'.format(item[1][0] + " " +item[1][1],item[0]))
    f.write("\n")

In [22]:
f.close()